In [1]:
from collections import defaultdict
from pathlib import Path
import torch
torch.set_grad_enabled(False)

from mycv.datasets.improb import simple_evaluate

weights_root = Path.cwd() / '../mycv/weights/my-vaes'

In [2]:
from mycv.models.vae.qres.library import dh_64s4x
weights_dir = weights_root / 'dh-64s4x'

stats = defaultdict(list)
for lmb in [16, 32, 64, 128, 256, 512, 1024, 2048]:
    # initialize model
    model = dh_64s4x(lmb=lmb)
    model = model.cuda()
    model.eval()

    wpath = weights_dir / f'dh_64s4x-lmb{lmb}/last_ema.pt'
    msd = torch.load(wpath)['model']
    model.load_state_dict(msd)
    model.compress_mode()

    results = simple_evaluate(model, dataset_name='kodak', batch_size=1)
    # print(results)
    for k,v in results.items():
        stats[k].append(v)

for k, vlist in stats.items():
    vlist_str = ', '.join([f'{v:.12f}'[:12] for v in vlist])
    print(f'{k:<12s} = [{vlist_str}]')

Scanning through images in d:\datasets\improcessing\kodak...


24it [00:00, ?it/s]


Evaluating <class 'mycv.models.vae.qres.qresvae.HierarchicalVAE'>, device=cuda:0, dtype=torch.float32
batch_size=1, num_workers=0


total_num=24, loss=0.1095, kl=0.04148, mse=0.06806, bppix=0.1796, psnr=30.02, ms-ssim=0.9391, real-bppix=0.184, real-psnr=30.02,: 100%|██████████| 24/24 [00:07<00:00,  3.01it/s] 


Scanning through images in d:\datasets\improcessing\kodak...


...

Scanning through images in d:\datasets\improcessing\kodak...


24it [00:00, 23956.04it/s]


Evaluating <class 'mycv.models.vae.qres.qresvae.HierarchicalVAE'>, device=cuda:0, dtype=torch.float32
batch_size=1, num_workers=0


total_num=24, loss=0.849, kl=0.5435, mse=0.3055, bppix=2.352, psnr=44.31, ms-ssim=0.9976, real-bppix=2.357, real-psnr=44.31,: 100%|██████████| 24/24 [00:06<00:00,  3.80it/s] 

loss         = [0.1095452358, 0.1550013646, 0.2120844858, 0.2918679155, 0.3799289309, 0.4934807183, 0.6517012727, 0.8489855751]
kl           = [0.0414848763, 0.0685851799, 0.1034742329, 0.1553431345, 0.2194844863, 0.2963687873, 0.4030155725, 0.5435158262]
mse          = [0.0680603586, 0.0864161841, 0.1086102531, 0.1365247843, 0.1604444465, 0.1971119263, 0.2486856989, 0.3054697544]
bppix        = [0.1795500761, 0.2968424967, 0.4478452883, 0.6723383096, 0.9499475398, 1.2827093394, 1.7442857038, 2.3523827615]
psnr         = [30.017866015, 31.973801732, 33.887499862, 35.849589026, 38.143325396, 40.229316274, 42.210288685, 44.313577042]
ms-ssim      = [0.9391223390, 0.9624467889, 0.9753316640, 0.9841089397, 0.9907682786, 0.9940640007, 0.9960501591, 0.9975985462]
real-bppix   = [0.1839658949, 0.3018018934, 0.4525799221, 0.6759821573, 0.9546271430, 1.2871958414, 1.7485911051, 2.3573125203]
real-psnr    = [30.017866015, 31.973801732, 33.887499862, 35.849589026, 38.143325396, 40.229316274, 42.2

In [3]:
from mycv.models.vae.qres.library import dhx_celeb64s4_ms

weights_dir = weights_root / 'dhx-celeb64-ms'

stats = defaultdict(list)
for lmb in [2, 4, 8, 16, 32, 64]:
    # initialize model
    model = dhx_celeb64s4_ms(lmb=lmb)
    model = model.cuda()
    model.eval()

    wpath = weights_dir / f'dhx-celeb64s4-ms-lmb{lmb}/last_ema.pt'
    msd = torch.load(wpath)['model']
    model.load_state_dict(msd)
    # model.compress_mode()

    results = simple_evaluate(model, dataset_name='celeba-test64', batch_size=128, workers=2)
    # print(results)
    for k,v in results.items():
        stats[k].append(v)

for k, vlist in stats.items():
    vlist_str = ', '.join([f'{v:.12f}'[:12] for v in vlist])
    print(f'{k:<12s} = [{vlist_str}]')

Scanning through images in d:\datasets\celeba\test64...


19962it [00:00, 166661.75it/s]


Evaluating <class 'mycv.models.vae.qres.qresvae.HierarchicalVAE'>, device=cuda:0, dtype=torch.float32
batch_size=128, num_workers=2


total_num=19962, loss=0.08942, kl=0.03688, 1-ms=0.05254, bppix=0.1596, psnr=25.18, ms-ssim=0.9781,: 100%|██████████| 156/156 [00:26<00:00,  5.94it/s]


Scanning through images in d:\datasets\celeba\test64...


...

Scanning through images in d:\datasets\celeba\test64...


19962it [00:00, 297681.16it/s]


Evaluating <class 'mycv.models.vae.qres.qresvae.HierarchicalVAE'>, device=cuda:0, dtype=torch.float32
batch_size=128, num_workers=2


total_num=19962, loss=0.4206, kl=0.2093, 1-ms=0.2112, bppix=0.906, psnr=31.64, ms-ssim=0.9972,: 100%|██████████| 156/156 [00:13<00:00, 11.17it/s] 

loss         = [0.0894202955, 0.1189151866, 0.1673885759, 0.2250479961, 0.3020993618, 0.4205616143]
kl           = [0.0368799307, 0.0589234038, 0.0878725139, 0.1192580291, 0.1613483537, 0.2093272806]
1-ms         = [0.0525403637, 0.0599917818, 0.0795160615, 0.1057899663, 0.1407510069, 0.2112343309]
bppix        = [0.1596194794, 0.2550255077, 0.3803197203, 0.5161589018, 0.6983294096, 0.9059862892]
psnr         = [25.183222741, 27.029023198, 28.283667112, 29.437971485, 30.759501795, 31.638951780]
ms-ssim      = [0.9780782941, 0.9872698184, 0.9916148896, 0.9943813795, 0.9962763363, 0.9972369268]
